In [1]:
import torch

In [2]:
t = torch.rand(3, 4)

In [4]:
t.permute(0,-1)

tensor([[0.2483, 0.7390, 0.8582],
        [0.3215, 0.8870, 0.2871],
        [0.2887, 0.5664, 0.2973],
        [0.2791, 0.4494, 0.9182]])

In [6]:
t.ndim

2